In [1]:
import numpy as np
import pandas as pd
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
train = pd.read_csv('C:/Users/KimSeongHoon/OneDrive/바탕 화면/competition_data/train.csv')
test = pd.read_csv('C:/Users/KimSeongHoon/OneDrive/바탕 화면/competition_data/test.csv')
submission = pd.read_csv('C:/Users/KimSeongHoon/OneDrive/바탕 화면/competition_data/sample_submission.csv')
train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 70 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   index         15000 non-null  int64  
 1   Q1            14959 non-null  float64
 2   Q2            14931 non-null  float64
 3   Q3            14950 non-null  float64
 4   Q4            14929 non-null  float64
 5   Q5            14962 non-null  float64
 6   Q6            14952 non-null  float64
 7   Q7            14924 non-null  float64
 8   Q8            14952 non-null  float64
 9   Q9            14944 non-null  float64
 10  Q10           14928 non-null  float64
 11  Q11           14941 non-null  float64
 12  Q12           14933 non-null  float64
 13  Q13           14960 non-null  float64
 14  Q14           14964 non-null  float64
 15  Q15           14955 non-null  float64
 16  Q16           14967 non-null  float64
 17  Q17           14963 non-null  float64
 18  Q18           14937 non-nu

In [4]:
target = train['nerdiness']
train.drop(['nerdiness'], axis=1, inplace=True)
all_data = pd.concat([train, test], axis=0)
print(all_data.shape)
all_data.head()
all_data = all_data.fillna(0)

(50452, 69)


In [5]:
all_data.loc[all_data['gender'] == 0, 'gender'] = 2
all_data.loc[all_data['married'] == 0, 'married'] = 1
all_data.loc[all_data['ASD'] == 0, 'ASD'] = 2
all_data.loc[all_data['education'] == 0, 'education'] = all_data['education'].mode()[0]
all_data.loc[all_data['voted'] == 0, 'voted'] = all_data['voted'].mode()[0]
all_data.loc[all_data['urban'] == 0, 'urban'] = all_data['urban'].mode()[0]
all_data.loc[all_data['orientation'] == 0, 'orientation'] = all_data['orientation'].mode()[0]
all_data.loc[all_data['religion'] == 0, 'religion'] = all_data['religion'].mode()[0]
all_data.loc[all_data['engnat'] == 0, 'engnat'] = all_data['engnat'].mode()[0]
all_data.loc[all_data['hand'] == 0, 'hand'] = all_data['hand'].mode()[0]
all_data.loc[all_data['familysize'] > 7, 'familysize'] = all_data['familysize'].mode()[0]
all_data.loc[all_data['age'] > 100, 'age'] = all_data['familysize'].mode()[0]
all_data.loc[all_data['Q1'] == 0, 'Q1'] = all_data['Q1'].mode()[0]
all_data.loc[all_data['Q2'] == 0, 'Q2'] = all_data['Q2'].mode()[0]
all_data.loc[all_data['Q3'] == 0, 'Q3'] = all_data['Q3'].mode()[0]
all_data.loc[all_data['Q4'] == 0, 'Q4'] = all_data['Q4'].mode()[0]
all_data.loc[all_data['Q5'] == 0, 'Q5'] = all_data['Q5'].mode()[0]
all_data.loc[all_data['Q6'] == 0, 'Q6'] = all_data['Q6'].mode()[0]
all_data.loc[all_data['Q7'] == 0, 'Q7'] = all_data['Q7'].mode()[0]
all_data.loc[all_data['Q8'] == 0, 'Q8'] = all_data['Q8'].mode()[0]
all_data.loc[all_data['Q9'] == 0, 'Q9'] = all_data['Q9'].mode()[0]
all_data.loc[all_data['Q10'] == 0, 'Q10'] = all_data['Q10'].mode()[0]
all_data.loc[all_data['Q11'] == 0, 'Q11'] = all_data['Q11'].mode()[0]
all_data.loc[all_data['Q12'] == 0, 'Q12'] = all_data['Q12'].mode()[0]
all_data.loc[all_data['Q13'] == 0, 'Q13'] = all_data['Q13'].mode()[0]
all_data.loc[all_data['Q14'] == 0, 'Q14'] = all_data['Q14'].mode()[0]
all_data.loc[all_data['Q15'] == 0, 'Q15'] = all_data['Q15'].mode()[0]
all_data.loc[all_data['Q16'] == 0, 'Q16'] = all_data['Q16'].mode()[0]
all_data.loc[all_data['Q17'] == 0, 'Q17'] = all_data['Q17'].mode()[0]
all_data.loc[all_data['Q18'] == 0, 'Q18'] = all_data['Q18'].mode()[0]
all_data.loc[all_data['Q19'] == 0, 'Q19'] = all_data['Q19'].mode()[0]
all_data.loc[all_data['Q20'] == 0, 'Q20'] = all_data['Q20'].mode()[0]
all_data.loc[all_data['Q21'] == 0, 'Q21'] = all_data['Q21'].mode()[0]
all_data.loc[all_data['Q22'] == 0, 'Q22'] = all_data['Q22'].mode()[0]
all_data.loc[all_data['Q23'] == 0, 'Q23'] = all_data['Q23'].mode()[0]
all_data.loc[all_data['Q24'] == 0, 'Q24'] = all_data['Q24'].mode()[0]
all_data.loc[all_data['Q25'] == 0, 'Q25'] = all_data['Q25'].mode()[0]
all_data.loc[all_data['Q26'] == 0, 'Q26'] = all_data['Q26'].mode()[0]

In [6]:
train = all_data.iloc[:train.shape[0]]
test = all_data.iloc[train.shape[0]:]
print(train)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [7]:
all_data.isnull().sum()
all_data['hand'].value_counts()

1.0    43180
2.0     5142
3.0     2130
Name: hand, dtype: int64

In [8]:
country_map = {i: c for c, i in enumerate(all_data['country'].unique())}

In [9]:
print(type(country_map))

print((country_map))

<class 'dict'>
{'USA': 0, 'NLD': 1, 'ITA': 2, 'ARE': 3, 'DNK': 4, 'GRC': 5, 'QAT': 6, 'GBR': 7, 'ROU': 8, 'HUN': 9, 'AUS': 10, 'POL': 11, 'CAN': 12, 'ARG': 13, 'FRA': 14, 'DEU': 15, 'UKR': 16, 'BGR': 17, 'PAK': 18, 'EST': 19, 'PHL': 20, 'SVN': 21, 0: 22, 'SRB': 23, 'NZL': 24, 'SWE': 25, 'MAR': 26, 'IDN': 27, 'HKG': 28, 'LUX': 29, 'CHE': 30, 'MYS': 31, 'RUS': 32, 'BRA': 33, 'NOR': 34, 'PRT': 35, 'MEX': 36, 'IND': 37, 'VNM': 38, 'AUT': 39, 'FIN': 40, 'ESP': 41, 'GUM': 42, 'CHL': 43, 'SGP': 44, 'HND': 45, 'KOR': 46, 'COL': 47, 'IRL': 48, 'CYP': 49, 'BEL': 50, 'AZE': 51, 'IRQ': 52, 'SVK': 53, 'TUR': 54, 'NGA': 55, 'ZAF': 56, 'JPN': 57, 'ISR': 58, 'URY': 59, 'ISL': 60, 'GHA': 61, 'CZE': 62, 'KEN': 63, 'ECU': 64, 'EGY': 65, 'JOR': 66, 'CRI': 67, 'THA': 68, 'PER': 69, 'NPL': 70, 'SAU': 71, 'MKD': 72, 'DOM': 73, 'PRI': 74, 'TTO': 75, 'CUW': 76, 'BGD': 77, 'LTU': 78, 'HRV': 79, 'IRN': 80, 'GRL': 81, 'SDN': 82, 'KWT': 83, 'LVA': 84, 'ALB': 85, 'MDV': 86, 'CHN': 87, 'VEN': 88, 'MUS': 89, 'JAM': 9

In [10]:
print((country_map.keys()))

dict_keys(['USA', 'NLD', 'ITA', 'ARE', 'DNK', 'GRC', 'QAT', 'GBR', 'ROU', 'HUN', 'AUS', 'POL', 'CAN', 'ARG', 'FRA', 'DEU', 'UKR', 'BGR', 'PAK', 'EST', 'PHL', 'SVN', 0, 'SRB', 'NZL', 'SWE', 'MAR', 'IDN', 'HKG', 'LUX', 'CHE', 'MYS', 'RUS', 'BRA', 'NOR', 'PRT', 'MEX', 'IND', 'VNM', 'AUT', 'FIN', 'ESP', 'GUM', 'CHL', 'SGP', 'HND', 'KOR', 'COL', 'IRL', 'CYP', 'BEL', 'AZE', 'IRQ', 'SVK', 'TUR', 'NGA', 'ZAF', 'JPN', 'ISR', 'URY', 'ISL', 'GHA', 'CZE', 'KEN', 'ECU', 'EGY', 'JOR', 'CRI', 'THA', 'PER', 'NPL', 'SAU', 'MKD', 'DOM', 'PRI', 'TTO', 'CUW', 'BGD', 'LTU', 'HRV', 'IRN', 'GRL', 'SDN', 'KWT', 'LVA', 'ALB', 'MDV', 'CHN', 'VEN', 'MUS', 'JAM', 'TWN', 'LBN', 'TUN', 'BIH', 'PNG', 'GUF', 'MDG', 'DZA', 'BRN', 'ALA', 'VGB', 'SLV', 'OMN', 'PRY', 'LBY', 'GEO', 'BWA', 'FRO', 'BHR', 'ZMB', 'UGA', 'NAM', 'KAZ', 'VIR', 'MLT', 'ARM', 'BHS', 'MAC', 'KHM', 'MNP', 'SYR', 'MNG', 'MOZ', 'FJI', 'AFG', 'FSM', 'AGO', 'MWI', 'SSD', 'GTM', 'LAO', 'GUY', 'RWA', 'PAN', 'BLR', 'BRB', 'ETH', 'GRD', 'MNE', 'NIC', 'SLE',

In [11]:
print((country_map.values()))

dict_values([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159])


In [12]:
print(type(train['country']))
# print(train['country'])

<class 'pandas.core.series.Series'>


In [13]:
train['country'] = train['country'].map(country_map)
test['country'] = test['country'].map(country_map)

In [14]:
print(train['country'])

0          0
1          0
2          1
3          0
4          2
5          0
6          0
7          1
8          0
9          3
10         4
11         5
12         0
13         0
14         0
15         6
16         7
17         8
18         7
19         0
20         0
21         9
22         0
23         0
24        10
25         0
26        10
27         0
28        11
29         0
30         0
31        12
32        10
33        10
34         0
35        13
36         0
37        14
38         2
39         0
40         7
41         1
42        10
43         0
44         0
45        15
46         2
47         7
48         0
49         0
50         0
51         0
52        16
53         2
54        17
55        18
56        19
57         0
58         0
59        10
60        20
61         0
62         0
63         0
64         1
65        21
66        22
67        23
68        24
69         0
70        15
71         0
72        10
73         0
74        25
75        12
76        10

In [15]:
train['country'].isnull().sum()

0

In [16]:
print(pd.DataFrame(train.isnull().sum()))
print(pd.DataFrame(test.isnull().sum()))

              0
index         0
Q1            0
Q2            0
Q3            0
Q4            0
Q5            0
Q6            0
Q7            0
Q8            0
Q9            0
Q10           0
Q11           0
Q12           0
Q13           0
Q14           0
Q15           0
Q16           0
Q17           0
Q18           0
Q19           0
Q20           0
Q21           0
Q22           0
Q23           0
Q24           0
Q25           0
Q26           0
country       0
introelapse   0
testelapse    0
surveyelapse  0
TIPI1         0
TIPI2         0
TIPI3         0
TIPI4         0
TIPI5         0
TIPI6         0
TIPI7         0
TIPI8         0
TIPI9         0
TIPI10        0
VCL1          0
VCL2          0
VCL3          0
VCL4          0
VCL5          0
VCL6          0
VCL7          0
VCL8          0
VCL9          0
VCL10         0
VCL11         0
VCL12         0
VCL13         0
VCL14         0
VCL15         0
VCL16         0
education     0
urban         0
gender        0
engnat        0
age     

In [17]:
train = train.drop(['index'],axis = 1)
test = test.drop(['index'],axis = 1)

In [18]:
x_train = train.loc[:, 'Q1':'ASD']
y_train = target

x_test = test.loc[:, 'Q1':'ASD']

In [19]:
print(pd.DataFrame(train.isnull().sum()))
print(pd.DataFrame(test.isnull().sum()))

              0
Q1            0
Q2            0
Q3            0
Q4            0
Q5            0
Q6            0
Q7            0
Q8            0
Q9            0
Q10           0
Q11           0
Q12           0
Q13           0
Q14           0
Q15           0
Q16           0
Q17           0
Q18           0
Q19           0
Q20           0
Q21           0
Q22           0
Q23           0
Q24           0
Q25           0
Q26           0
country       0
introelapse   0
testelapse    0
surveyelapse  0
TIPI1         0
TIPI2         0
TIPI3         0
TIPI4         0
TIPI5         0
TIPI6         0
TIPI7         0
TIPI8         0
TIPI9         0
TIPI10        0
VCL1          0
VCL2          0
VCL3          0
VCL4          0
VCL5          0
VCL6          0
VCL7          0
VCL8          0
VCL9          0
VCL10         0
VCL11         0
VCL12         0
VCL13         0
VCL14         0
VCL15         0
VCL16         0
education     0
urban         0
gender        0
engnat        0
age           0
hand    

In [20]:
X_train, X_test, y_train, y_test =\
    train_test_split(x_train, y_train, test_size=0.3, random_state=2020)

In [21]:
rf_clf = RandomForestClassifier()
rf_clf.fit(X_train, y_train)
rf_pred = rf_clf.predict_proba(X_test)[:, 1]
roc_auc_score(y_test, rf_pred)

0.8629527948815157

In [25]:
rf_clf = RandomForestClassifier(n_estimators=700)
rf_clf.fit(X_train, y_train)
rf_pred = rf_clf.predict_proba(X_test)[:, 1]
roc_auc_score(y_test, rf_pred)

0.8683628064623482

In [26]:
lgbm_clf = LGBMClassifier()
lgbm_clf.fit(X_train, y_train)
lgbm_pred = lgbm_clf.predict_proba(X_test)[:, 1]
roc_auc_score(y_test, lgbm_pred)

0.8226529549289808

In [27]:
lgbm_clf = LGBMClassifier(
            n_estimators=10000,
            learning_rate=0.01,
            random_state=100,
            max_depth=10,
            num_leaves=600,
            min_child_samples=10,
            subsample=0.80,
            colsample_bytree=0.50,
            max_bin=450,
            reg_lambda=0.07,
            reg_alpha=0.28
        )
lgbm_clf.fit(X_train, y_train)
lgbm_pred = lgbm_clf.predict_proba(X_test)[:, 1]
roc_auc_score(y_test, lgbm_pred)

0.8630870494909404

In [28]:
y_preds = 0.3 * lgbm_pred + 0.7 * rf_pred
roc_auc_score(y_test, y_preds)

0.8695462433763567

In [29]:
lgbm_clf.fit(train, target)
lgbm_pred = lgbm_clf.predict_proba(test)[:, 1]
rf_clf.fit(train, target)
rf_pred = rf_clf.predict_proba(test)[:, 1]
y_pred = 0.4 * lgbm_pred + 0.6 * rf_pred
submission['nerdiness'] = y_pred
submission.to_csv('test.csv', index=False)